In [1]:
from sagemaker.model_monitor import DataCaptureConfig

# Please fill in the following for enabling data capture
s3_capture_upload_path = f"s3://sagemaker-us-west-2-846634201516/monitoring/"  # example: s3://bucket-name/path/to/endpoint-data-capture/


In [6]:
import sagemaker
import boto3
# SageMaker session
sess = sagemaker.Session()
# get session bucket name
bucket = sess.default_bucket()
# bucket prefix or the subfolder for everything we produce
prefix = "music-recommendation"
# get sagemaker role
sagemaker_role = sagemaker.get_execution_role()
# s3 client
s3_client = boto3.client("s3")

region = boto3.Session().region_name
boto_session = boto3.Session(region_name=region)


sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)
sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session, sagemaker_client=sagemaker_client
)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [7]:
%%time
endpoint_name = "CustomerChurn"
predictor = sagemaker.predictor.Predictor(
    endpoint_name=endpoint_name, sagemaker_session=sagemaker_session
)

# Change parameters as you would like - adjust sampling percentage,
#  chose to capture request or response or both
data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=s3_capture_upload_path,
    kms_key_id=None,
    capture_options=["REQUEST", "RESPONSE"],
    csv_content_types=["text/csv"],
    json_content_types=["application/json"],
)

# Now it is time to apply the new configuration and wait for it to be applied
predictor.update_data_capture_config(data_capture_config=data_capture_config)
sess.wait_for_endpoint(endpoint=endpoint_name)


------!!CPU times: user 79.8 ms, sys: 1.65 ms, total: 81.4 ms
Wall time: 3min 1s


{'EndpointName': 'CustomerChurn',
 'EndpointArn': 'arn:aws:sagemaker:us-west-2:846634201516:endpoint/customerchurn',
 'EndpointConfigName': 'CustomerChurn-2024-01-25-04-39-15-884',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'DeployedImages': [{'SpecifiedImage': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
     'ResolvedImage': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost@sha256:0c8f830ac408e6dee08445fb60392e9c3f05f790a4b3c07ec22327c08f75bcbf',
     'ResolutionTime': datetime.datetime(2024, 1, 25, 4, 39, 18, 560000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1}],
 'DataCaptureConfig': {'EnableCapture': True,
  'CaptureStatus': 'Started',
  'CurrentSamplingPercentage': 100,
  'DestinationS3Uri': 's3://sagemaker-us-west-2-846634201516/monitoring/'},
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2024, 1, 25, 4, 33, 19, 148000, tzinfo=tzlocal()),

In [9]:
##'s3://bucketname/path/to/baseline/data' - Where your validation data is
baseline_data_uri = f"s3://sagemaker-us-west-2-846634201516/ml_deploy/validation"
##'s3://bucketname/path/to/baseline/data' - Where the results are to be stored in
baseline_results_uri = f"s3://sagemaker-us-west-2-846634201516/monitoring/baseline/results"

print("Baseline data uri: {}".format(baseline_data_uri))
print("Baseline results uri: {}".format(baseline_results_uri))

Baseline data uri: s3://sagemaker-us-west-2-846634201516/ml_deploy/validation
Baseline results uri: s3://sagemaker-us-west-2-846634201516/monitoring/baseline/results


In [10]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker import get_execution_role
import datetime

role = get_execution_role(sagemaker_session=sess)
prefix = "customerchurn"

datetime_stamp = datetime.datetime.now().strftime("%Y-%m-%d-%H%M%S")

my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    base_job_name=f"{prefix}-monitor-{datetime_stamp}",
)

In [11]:
%%time

monitor_baseline = my_default_monitor.suggest_baseline(
    baseline_dataset=baseline_data_uri,
    dataset_format=DatasetFormat.csv(header=False),
    output_s3_uri=baseline_results_uri,
    job_name=f"{prefix}-monitor-baseline-{datetime_stamp}",
    wait=True,
)


Job Name:  customerchurn-monitor-baseline-2024-01-25-044606
Inputs:  [{'InputName': 'baseline_dataset_input', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-846634201516/ml_deploy/validation', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-846634201516/monitoring/baseline/results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
...............................2024-01-25 04:51:04.522789: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-25 04:51:04.522829: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dle

In [12]:
from time import gmtime, strftime
import boto3

client = boto3.client("sagemaker")


def get_last_processing_job():
    response = client.list_processing_jobs(
        NameContains=f"{prefix}-monitor-baseline-{datetime_stamp}",
        StatusEquals="Completed",
        SortBy="CreationTime",
        SortOrder="Descending",
        MaxResults=20,
    )
    pprint.pprint(response["ProcessingJobSummaries"][0])
    return response["ProcessingJobSummaries"][0]["ProcessingJobName"]

In [14]:
from sagemaker.processing import ProcessingJob
from sagemaker.estimator import Estimator
from sagemaker.model_monitor.model_monitoring import ModelMonitor
import pprint
my_default_monitor_name = get_last_processing_job()


{'CreationTime': datetime.datetime(2024, 1, 25, 4, 46, 9, 564000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 1, 25, 4, 53, 8, 838000, tzinfo=tzlocal()),
 'ProcessingEndTime': datetime.datetime(2024, 1, 25, 4, 53, 8, 529000, tzinfo=tzlocal()),
 'ProcessingJobArn': 'arn:aws:sagemaker:us-west-2:846634201516:processing-job/customerchurn-monitor-baseline-2024-01-25-044606',
 'ProcessingJobName': 'customerchurn-monitor-baseline-2024-01-25-044606',
 'ProcessingJobStatus': 'Completed'}


In [15]:
my_default_monitor_reload = ProcessingJob.from_processing_name(sess, my_default_monitor_name)

response = client.describe_processing_job(ProcessingJobName=my_default_monitor_name)
pprint.pprint(response)

{'AppSpecification': {'ImageUri': '159807026194.dkr.ecr.us-west-2.amazonaws.com/sagemaker-model-monitor-analyzer'},
 'CreationTime': datetime.datetime(2024, 1, 25, 4, 46, 9, 564000, tzinfo=tzlocal()),
 'Environment': {'dataset_format': '{"csv": {"header": false, '
                                   '"output_columns_position": "START"}}',
                 'dataset_source': '/opt/ml/processing/input/baseline_dataset_input',
                 'output_path': '/opt/ml/processing/output',
                 'publish_cloudwatch_metrics': 'Disabled'},
 'ExitMessage': 'Completed: Job completed successfully with no violations.',
 'LastModifiedTime': datetime.datetime(2024, 1, 25, 4, 53, 8, 838000, tzinfo=tzlocal()),
 'ProcessingEndTime': datetime.datetime(2024, 1, 25, 4, 53, 8, 529000, tzinfo=tzlocal()),
 'ProcessingInputs': [{'AppManaged': False,
                       'InputName': 'baseline_dataset_input',
                       'S3Input': {'LocalPath': '/opt/ml/processing/input/baseline_dataset_

In [16]:
import pandas as pd

baseline_job = my_default_monitor.latest_baselining_job
schema_df = pd.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df.head(10)

,name,inferred_type,numerical_statistics.common.num_present,numerical_statistics.common.num_missing,numerical_statistics.mean,numerical_statistics.sum,numerical_statistics.std_dev,numerical_statistics.min,numerical_statistics.max,numerical_statistics.distribution.kll.buckets,numerical_statistics.distribution.kll.sketch.parameters.c,numerical_statistics.distribution.kll.sketch.parameters.k,numerical_statistics.distribution.kll.sketch.data
0,_c0,Fractional,1050,0,0.516190,542.000000,0.499738,0.000000,1.000000,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,_c1,Fractional,1050,0,102.530476,107657.000000,57.034288,1.000000,200.000000,"[{'lower_bound': 1.0, 'upper_bound': 20.9, 'co...",0.64,2048.0,"[[1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 3.0, 3.0, 3.0,..."
2,_c2,Fractional,1050,0,229.809524,241300.000000,276.089982,0.000000,1300.000000,"[{'lower_bound': 0.0, 'upper_bound': 130.0, 'c...",0.64,2048.0,"[[0.0, 200.0, 0.0, 0.0, 200.0, 400.0, 0.0, 100..."
3,_c3,Fractional,1050,0,5.679779,5963.768387,3.340890,0.002911,16.504147,"[{'lower_bound': 0.0029108563531714533, 'upper...",0.64,2048.0,"[[1.0526538076293468, 1.7525156498596337, 0.02..."
4,_c4,Fractional,1050,0,3.501905,3677.000000,1.701119,0.000000,9.000000,"[{'lower_bound': 0.0, 'upper_bound': 0.9, 'cou...",0.64,2048.0,"[[2.0, 5.0, 3.0, 5.0, 4.0, 5.0, 0.0, 6.0, 6.0,..."
5,_c5,Fractional,1050,0,5.039082,5291.035826,2.121572,0.036409,12.133130,"[{'lower_bound': 0.03640900288750437, 'upper_b...",0.64,2048.0,"[[0.6993549113167711, 7.0170936431980255, 5.40..."
6,_c6,Fractional,1050,0,3.107619,3263.000000,2.532990,0.000000,13.000000,"[{'lower_bound': 0.0, 'upper_bound': 1.3, 'cou...",0.64,2048.0,"[[2.0, 3.0, 5.0, 7.0, 8.0, 7.0, 3.0, 5.0, 1.0,..."
7,_c7,Fractional,1050,0,4.123123,4329.278947,1.635218,0.013675,9.193927,"[{'lower_bound': 0.0136750570626889, 'upper_bo...",0.64,2048.0,"[[5.073754840111169, 1.3198145052020855, 3.907..."
8,_c8,Fractional,1050,0,220.666667,231700.000000,96.789085,0.000000,550.000000,"[{'lower_bound': 0.0, 'upper_bound': 55.0, 'co...",0.64,2048.0,"[[200.0, 150.0, 400.0, 300.0, 250.0, 300.0, 20..."
9,_c9,Fractional,1050,0,5.008737,5259.174012,1.009469,1.648514,8.181336,"[{'lower_bound': 1.6485135544570038, 'upper_bo...",0.64,2048.0,"[[5.823936374928548, 5.2857977513777135, 4.111..."


In [17]:
constraints_df = pd.json_normalize(baseline_job.suggested_constraints().body_dict["features"])
constraints_df.head(10)

,name,inferred_type,completeness,num_constraints.is_non_negative
0,_c0,Fractional,1.0,True
1,_c1,Fractional,1.0,True
2,_c2,Fractional,1.0,True
3,_c3,Fractional,1.0,True
4,_c4,Fractional,1.0,True
5,_c5,Fractional,1.0,True
6,_c6,Fractional,1.0,True
7,_c7,Fractional,1.0,True
8,_c8,Fractional,1.0,True
9,_c9,Fractional,1.0,True
